In [ ]:
import os
import random
import requests
import shutil
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D, Conv2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [ ]:
train_dir = #refer to data preprocessing script to set this path
test_dir =  #refer to data preprocessing script to set this path

In [ ]:
train_gen = ImageDataGenerator(
        rescale = 1./255,
        featurewise_center = False,  
        samplewise_center = False, 
        featurewise_std_normalization = False,  
        samplewise_std_normalization = False,  
        zca_whitening = False,  
        rotation_range = 10,  
        zoom_range = 0.1, 
        width_shift_range = 0.2,  
        height_shift_range = 0.2, 
        horizontal_flip = True,  
        vertical_flip = False) 

test_gen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_generator = train_gen.flow_from_directory(
        train_dir,
        target_size=(100, 100),
        batch_size=16,
        )

validation_generator = test_gen.flow_from_directory(
        test_dir,
        target_size=(100, 100),
        batch_size=16,
        )

In [ ]:
base_model = VGG16(include_top = False,
                  input_shape = (100, 100, 3),
                  weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False
    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))
model.add(Dense(1024, activation = "relu"))
model.add((Dropout(0.5)))
model.add((Dense(120, activation = "softmax")))

model.summary()

In [ ]:
# a check point callback to save our best weights
filepath = #set path
checkpoint = ModelCheckpoint(filepath, 
                             monitor='val_accuracy', 
                             verbose=1, 
                             save_best_only=False, 
                             mode='max', 
                             save_weights_only=False)

# a reducing lr callback to reduce lr when val_loss doesn't increase
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                   patience=1, verbose=1, mode='min',
                                   min_delta=0.0001, cooldown=2, min_lr=1e-7)

# for early stop
early_stop = EarlyStopping(monitor="val_loss", mode="min", patience=10)


In [ ]:
model.compile(loss = 'categorical_crossentropy', 
              optimizer = 'adam',
             metrics = ['accuracy'])
model.summary()
history = model.fit_generator(
    train_generator,
    validation_data  = validation_generator,
    validation_steps = 1000,
    steps_per_epoch  = 1000,
    epochs = 50, 
    verbose = 1,
    callbacks=[checkpoint, reduce_lr, early_stop]
)

In [ ]:
def show_final_history(history):
    fig, ax = plt.subplots(1, 2, figsize=(15,5))
    ax[0].set_title('loss')
    ax[0].plot(history.epoch, history.history["loss"], label="Train loss")
    ax[0].plot(history.epoch, history.history["val_loss"], label="Validation loss")
    ax[1].set_title('acc')
    ax[1].plot(history.epoch, history.history["accuracy"], label="Train acc")
    ax[1].plot(history.epoch, history.history["val_accuracy"], label="Validation acc")
    ax[0].legend()
    ax[1].legend()

In [ ]:
show_final_history(history)

model_score = model.evaluate(validation_generator)
print("Model Test Loss:",model_score[0])
print("Model Test Accuracy:",model_score[1])